# Linear Elasticity

![](linear_elasticity.svg)

*Figure 1*: Linear elastically deformed 1mm $\times$ 1mm Ferrite logo.



## Implementation

The following code is based on the [Linear Elasticity](https://ferrite-fem.github.io/Ferrite.jl/stable/tutorials/linear_elasticity/) tutorial from the Ferrite.jl documentation, with some comments removed for brevity.
There are two main modifications:

1. Fourth-order `Lagrange` shape functions are used for field approximation: `ip = Lagrange{RefTriangle,4}()^2`.
2. High-order quadrature points are used to accommodate the fourth-order shape functions: `qr = QuadratureRule{RefTriangle}(8)`.

In [1]:
using Ferrite, FerriteGmsh, FerriteMultigrid, AlgebraicMultigrid
using Downloads: download
using IterativeSolvers
using TimerOutputs

TimerOutputs.enable_debug_timings(AlgebraicMultigrid)
TimerOutputs.enable_debug_timings(FerriteMultigrid)

Emod = 200.0e3 # Young's modulus [MPa]
ν = 0.3        # Poisson's ratio [-]

Gmod = Emod / (2(1 + ν))  # Shear modulus
Kmod = Emod / (3(1 - 2ν)) # Bulk modulus

C = gradient(ϵ -> 2 * Gmod * dev(ϵ) + 3 * Kmod * vol(ϵ), zero(SymmetricTensor{2,2}))

function assemble_external_forces!(f_ext, dh, facetset, facetvalues, prescribed_traction)
    # Create a temporary array for the facet's local contributions to the external force vector
    fe_ext = zeros(getnbasefunctions(facetvalues))
    for facet in FacetIterator(dh, facetset)
        # Update the facetvalues to the correct facet number
        reinit!(facetvalues, facet)
        # Reset the temporary array for the next facet
        fill!(fe_ext, 0.0)
        # Access the cell's coordinates
        cell_coordinates = getcoordinates(facet)
        for qp in 1:getnquadpoints(facetvalues)
            # Calculate the global coordinate of the quadrature point.
            x = spatial_coordinate(facetvalues, qp, cell_coordinates)
            tₚ = prescribed_traction(x)
            # Get the integration weight for the current quadrature point.
            dΓ = getdetJdV(facetvalues, qp)
            for i in 1:getnbasefunctions(facetvalues)
                Nᵢ = shape_value(facetvalues, qp, i)
                fe_ext[i] += tₚ ⋅ Nᵢ * dΓ
            end
        end
        # Add the local contributions to the correct indices in the global external force vector
        assemble!(f_ext, celldofs(facet), fe_ext)
    end
    return f_ext
end

function assemble_cell!(ke, cellvalues, C)
    for q_point in 1:getnquadpoints(cellvalues)
        # Get the integration weight for the quadrature point
        dΩ = getdetJdV(cellvalues, q_point)
        for i in 1:getnbasefunctions(cellvalues)
            # Gradient of the test function
            ∇Nᵢ = shape_gradient(cellvalues, q_point, i)
            for j in 1:getnbasefunctions(cellvalues)
                # Symmetric gradient of the trial function
                ∇ˢʸᵐNⱼ = shape_symmetric_gradient(cellvalues, q_point, j)
                ke[i, j] += (∇Nᵢ ⊡ C ⊡ ∇ˢʸᵐNⱼ) * dΩ
            end
        end
    end
    return ke
end

function assemble_global!(K, dh, cellvalues, C)
    # Allocate the element stiffness matrix
    n_basefuncs = getnbasefunctions(cellvalues)
    ke = zeros(n_basefuncs, n_basefuncs)
    # Create an assembler
    assembler = start_assemble(K)
    # Loop over all cells
    for cell in CellIterator(dh)
        # Update the shape function gradients based on the cell coordinates
        reinit!(cellvalues, cell)
        # Reset the element stiffness matrix
        fill!(ke, 0.0)
        # Compute element contribution
        assemble_cell!(ke, cellvalues, C)
        # Assemble ke into K
        assemble!(assembler, celldofs(cell), ke)
    end
    return K
end

function linear_elasticity_2d(C)
    logo_mesh = "logo.geo"
    asset_url = "https://raw.githubusercontent.com/Ferrite-FEM/Ferrite.jl/gh-pages/assets/"
    isfile(logo_mesh) || download(string(asset_url, logo_mesh), logo_mesh)

    grid = togrid(logo_mesh)
    addfacetset!(grid, "top", x -> x[2] ≈ 1.0) # facets for which x[2] ≈ 1.0 for all nodes
    addfacetset!(grid, "left", x -> abs(x[1]) < 1.0e-6)
    addfacetset!(grid, "bottom", x -> abs(x[2]) < 1.0e-6)

    dim = 2
    order = 4
    ip = Lagrange{RefTriangle,order}()^dim # vector valued interpolation
    ip_coarse = Lagrange{RefTriangle,1}()^dim

    qr = QuadratureRule{RefTriangle}(8)
    qr_face = FacetQuadratureRule{RefTriangle}(6)

    cellvalues = CellValues(qr, ip)
    facetvalues = FacetValues(qr_face, ip)

    dh = DofHandler(grid)
    add!(dh, :u, ip)
    close!(dh)

    dh_coarse = DofHandler(grid)
    add!(dh_coarse, :u, ip_coarse)
    close!(dh_coarse)

    ch = ConstraintHandler(dh)
    add!(ch, Dirichlet(:u, getfacetset(grid, "bottom"), (x, t) -> 0.0, 2))
    add!(ch, Dirichlet(:u, getfacetset(grid, "left"), (x, t) -> 0.0, 1))
    close!(ch)

    traction(x) = Vec(0.0, 20.0e3 * x[1])

    A = allocate_matrix(dh)
    assemble_global!(A, dh, cellvalues, C)

    b = zeros(ndofs(dh))
    assemble_external_forces!(b, dh, getfacetset(grid, "top"), facetvalues, traction)
    apply!(A, b, ch)

    return A, b, dh, dh_coarse, cellvalues, ch
end

linear_elasticity_2d (generic function with 1 method)

### Near Null Space (NNS)

In multigrid methods for problems with vector-valued unknowns, such as linear elasticity,
the near null space represents the low energy mode or the smooth error that needs to be captured
in the coarser grid when using SA-AMG (Smoothed Aggregation Algebraic Multigrid), more on the topic
can be found  in [schroder2010](@citet).

For 2D linear elasticity problems, the rigid body modes are:
1. Translation in the x-direction,
2. Translation in the y-direction,
3. Rotation about the z-axis (i.e., $x_3$): each point (x, y) is mapped to (-y, x).

The function `create_nns` constructs the NNS matrix `B ∈ ℝ^{n × 3}`, where `n` is the number of degrees of freedom (DOFs)
for the case of `p` = 1 (i.e., linear interpolation), because `B` is only relevant for AMG.

In [2]:
function create_nns(dh, fieldname = first(dh.field_names))
    @assert length(dh.field_names) == 1 "Only a single field is supported for now."

    coords_flat = zeros(ndofs(dh))
    apply_analytical!(coords_flat, dh, fieldname, x -> x)
    coords = reshape(coords_flat, (length(coords_flat) ÷ 2, 2))

    grid = dh.grid
    B = zeros(Float64, ndofs(dh), 3)
    B[1:2:end, 1] .= 1 # x - translation
    B[2:2:end, 2] .= 1 # y - translation

    # in-plane rotation (x,y) → (-y,x)
    x = coords[:, 1]
    y = coords[:, 2]
    B[1:2:end, 3] .= -y
    B[2:2:end, 3] .= x

    return B
end

create_nns (generic function with 2 methods)

### Setup the linear elasticity problem
Load `FerriteMultigrid` to access the p-multigrid solver.

In [3]:
using FerriteMultigrid

Construct the linear elasticity problem with 2nd order polynomial shape functions.

In [4]:
A, b, dh, dh_coarse, cellvalues, ch = linear_elasticity_2d(C);

Info    : Reading 'logo.geo'...
Info    : Done reading 'logo.geo'
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 20%] Meshing curve 3 (Line)
Info    : [ 20%] Meshing curve 4 (Line)
Info    : [ 30%] Meshing curve 5 (Line)
Info    : [ 30%] Meshing curve 6 (Line)
Info    : [ 40%] Meshing curve 7 (Line)
Info    : [ 40%] Meshing curve 8 (Line)
Info    : [ 50%] Meshing curve 9 (Line)
Info    : [ 60%] Meshing curve 10 (Line)
Info    : [ 60%] Meshing curve 11 (Line)
Info    : [ 70%] Meshing curve 12 (Line)
Info    : [ 70%] Meshing curve 13 (Line)
Info    : [ 80%] Meshing curve 14 (Line)
Info    : [ 80%] Meshing curve 15 (Line)
Info    : [ 90%] Meshing curve 16 (Line)
Info    : [ 90%] Meshing curve 17 (Line)
Info    : [100%] Meshing curve 18 (Line)
Info    : Done meshing 1D (Wall 0.000961293s, CPU 0.000962s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : [ 20%] Meshing surface 2 

Construct the near null space (NNS) matrix

In [5]:
B = create_nns(dh_coarse)

208×3 Matrix{Float64}:
 1.0  0.0  -1.0
 0.0  1.0   0.801535
 1.0  0.0  -0.253172
 0.0  1.0   0.454964
 1.0  0.0  -0.612999
 0.0  1.0   0.900767
 1.0  0.0  -0.0858959
 0.0  1.0   0.417361
 1.0  0.0  -0.66085
 0.0  1.0   0.820318
 ⋮         
 0.0  1.0   0.11219
 1.0  0.0  -0.464069
 0.0  1.0   0.592985
 1.0  0.0  -0.450733
 0.0  1.0   0.301241
 1.0  0.0  -0.677392
 0.0  1.0   1.0
 1.0  0.0  -0.477795
 0.0  1.0   0.126586

> **Danger**
>
> Since NNS matrix is only relevant for AMG, and it is not used in the p-multigrid solver, therefore, `B` has to provided using linear field approximation (i.e., `p = 1`) when using AMG as the coarse solver, otherwise (e.g., using `Pinv` as the coarse solver), then we don't have to provide it.

Construct the finite element space $\mathcal{V}_{h,p = 4}$

In [6]:
fe_space = FESpace(dh, cellvalues, ch)

FESpace{Ferrite.DofHandler{2, Ferrite.Grid{2, Ferrite.Triangle, Float64}}, Ferrite.CellValues{Ferrite.FunctionValues{1, Ferrite.VectorizedInterpolation{2, Ferrite.RefTriangle, 4, Ferrite.Lagrange{Ferrite.RefTriangle, 4}}, Matrix{Tensors.Vec{2, Float64}}, Matrix{Tensors.Tensor{2, 2, Float64, 4}}, Matrix{Tensors.Tensor{2, 2, Float64, 4}}, Nothing, Nothing}, Ferrite.GeometryMapping{1, Ferrite.Lagrange{Ferrite.RefTriangle, 1}, Matrix{Float64}, Matrix{Tensors.Vec{2, Float64}}, Nothing}, Ferrite.QuadratureRule{Ferrite.RefTriangle, Vector{Float64}, Vector{Tensors.Vec{2, Float64}}}, Vector{Float64}}, Ferrite.ConstraintHandler{Ferrite.DofHandler{2, Ferrite.Grid{2, Ferrite.Triangle, Float64}}, Float64}}(Ferrite.DofHandler{2, Ferrite.Grid{2, Ferrite.Triangle, Float64}}(Ferrite.SubDofHandler{Ferrite.DofHandler{2, Ferrite.Grid{2, Ferrite.Triangle, Float64}}}[Ferrite.SubDofHandler{Ferrite.DofHandler{2, Ferrite.Grid{2, Ferrite.Triangle, Float64}}}(Ferrite.DofHandler{2, Ferrite.Grid{2, Ferrite.Triangl

### P-multigrid Configuration

In [7]:
reset_timer!()

pcoarse_solver = SmoothedAggregationCoarseSolver(; B)

SmoothedAggregationCoarseSolver{Tuple{}, Base.Pairs{Symbol, Matrix{Float64}, Nothing, @NamedTuple{B::Matrix{Float64}}}}((), Base.Pairs(:B => [1.0 0.0 -1.0; 0.0 1.0 0.801534880751; … ; 1.0 0.0 -0.4777949032272307; 0.0 1.0 0.1265859930873549]))

#### 0. CG as baseline

In [8]:
@timeit "CG" x_cg = IterativeSolvers.cg(A, b; maxiter = 1000, verbose=false)

2914-element Vector{Float64}:
 -0.009922773142129101
  0.027908992919798835
 -0.012273544906839417
  0.02778766287937867
 -0.01171609807787134
  0.03380248712544142
 -0.010480498291317988
  0.027964097065637683
 -0.011058650993627232
  0.027963704627201345
  ⋮
  0.02044737400079658
 -0.003425201397929082
  0.01966749967600462
 -0.006499317726805468
  0.027966501533483217
 -0.007055602277993196
  0.029169542096681003
 -0.007090577588019493
  0.029186469234192165

#### 1. Galerkin Coarsening Strategy

In [9]:
config_gal = pmultigrid_config(coarse_strategy = Galerkin())
@timeit "Galerkin only" x_gal, res_gal = FerriteMultigrid.solve(A, b,fe_space, config_gal; pcoarse_solver, verbose=false, log=true, rtol = 1e-10)

builder_gal = PMultigridPreconBuilder(fe_space, config_gal; pcoarse_solver)
@timeit "Build preconditioner" Pl_gal = builder_gal(A)[1]
@timeit "Galerkin CG" IterativeSolvers.cg(A, b; Pl = Pl_gal, maxiter = 1000, verbose=false)

2914-element Vector{Float64}:
 -0.00992277315074982
  0.027908992913412128
 -0.012273544888830971
  0.02778766291311703
 -0.011716098087832071
  0.03380248711993018
 -0.01048049829364379
  0.027964097067214727
 -0.01105865098999501
  0.027963704638106202
  ⋮
  0.02044737399268512
 -0.003425201420622597
  0.019667499668565236
 -0.00649931770467806
  0.02796650151137346
 -0.007055602255972188
  0.029169542075212395
 -0.007090577561606642
  0.029186469211926667

#### 2. Rediscretization Coarsening Strategy

In [10]:
# Rediscretization Coarsening Strategy
config_red = pmultigrid_config(coarse_strategy = Rediscretization(LinearElasticityMultigrid(C)))
@timeit "Rediscretization only" x_red, res_red = solve(A, b, fe_space, config_red; pcoarse_solver, log=true, rtol = 1e-10)

builder_red = PMultigridPreconBuilder(fe_space, config_red; pcoarse_solver)
@timeit "Build preconditioner" Pl_red = builder_red(A)[1]
@timeit "Rediscretization CG" IterativeSolvers.cg(A, b; Pl = Pl_red, maxiter = 1000, verbose=false)

print_timer(title = "Analysis with $(getncells(dh.grid)) elements", linechars = :ascii)

---------------------------------------------------------------------------------------
     Analysis with 174 elements               Time                    Allocations      
                                     -----------------------   ------------------------
          Tot / % measured:               3.04s /  84.0%            284MiB /  82.7%    

Section                      ncalls     time    %tot     avg     alloc    %tot      avg
---------------------------------------------------------------------------------------
Galerkin only                     1    1.53s   59.9%   1.53s    135MiB   57.7%   135MiB
  init                            1    1.18s   46.1%   1.18s    110MiB   46.7%   110MiB
    coarsen order                 1    657ms   25.7%   657ms   72.6MiB   30.9%  72.6MiB
    extend_hierarchy!             1    518ms   20.3%   518ms   35.9MiB   15.3%  35.9MiB
      build prolongator           1   11.1ms    0.4%  11.1ms   4.42MiB    1.9%  4.42MiB
        assembly               

### Test the solution

In [11]:
using Test
@testset "Linear Elasticity Example" begin
    println("Final residual with Galerkin coarsening: ", res_gal[end])
    @test A * x_gal ≈ b atol=1e-4
    println("Final residual with Rediscretization coarsening: ", res_red[end])
    @test A * x_red ≈ b atol=1e-4
end

Final residual with Galerkin coarsening: 3.348568026560081e-5
Final residual with Rediscretization coarsening: 3.316374143182177e-5
Test Summary:             | Pass  Total  Time
Linear Elasticity Example |    2      2  0.6s


Test.DefaultTestSet("Linear Elasticity Example", Any[], 2, false, false, true, 1.771278308387109e9, 1.771278308999951e9, false, "/home/runner/work/FerriteMultigrid.jl/FerriteMultigrid.jl/docs/src/tutorials/linear_elasticity.ipynb", Random.Xoshiro(0x07f9f19fc206ce53, 0x80de986faea45cd6, 0xa78ba20b95b7d93b, 0x7d8096f63d3cd8f6, 0x3ed405e4673511b6))

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*